# Importations et Configuration Initiale

In [1]:
import enoslib as en
import logging
import os
from pathlib import Path
import time

# --- Configuration de l'expérience ---

# Configurez le logging pour avoir un retour clair sur les étapes
logging.basicConfig(level=logging.INFO)

# ------------------------------------------------------------------
# MODIFIEZ CETTE LIGNE AVEC VOTRE NOM D'UTILISATEUR GRID'5000
G5K_USER = "wemenra"
# ------------------------------------------------------------------

G5K_SITE = "rennes"  # Site Grid'5000 à utiliser

# Nom du job pour la réservation sur Grid'5000
JOB_NAME = "K8sEnoslibDeploy"

# Clé SSH, par défaut enoslib cherche ~/.ssh/id_rsa
# Si vous en utilisez une autre, décommentez et modifiez la ligne suivante :
# en.set_config(ssh_key="~/.ssh/ma_cle_ssh")

# Création du chemin pour les sorties (par exemple, inventaire)
Path("inventory").mkdir(exist_ok=True)
OUTPUT_INVENTORY = "inventory/g5k_inventory.yaml"

print("Configuration initiale chargée.")
if G5K_USER == "VOTRE_LOGIN_G5K":
    print("\n⚠️  ATTENTION: N'oubliez pas de remplacer 'VOTRE_LOGIN_G5K' par votre nom d'utilisateur Grid'5000.")

[WARNING]: failed to patch stdout/stderr for fork-safety: 'OutStream' object
has no attribute 'buffer'
[WARNING]: failed to reconfigure stdout/stderr with custom encoding error
handler: 'OutStream' object has no attribute 'reconfigure'


Configuration initiale chargée.


# Définition des Ressources pour le Cluster

In [2]:
# CELLULE 2
# Définition de la configuration pour la connexion à Grid'5000
# en.set_provider(
#     en.G5k(
#         username=G5K_USER,
#         site=G5K_SITE,
#         job_name=JOB_NAME,
#     )
# )

# Définition des ressources à réserver
# 1 nœud master et 2 nœuds workers
# Tous les nœuds seront déployés avec Ubuntu 22.04
conf = (
en.G5kConf.from_settings(
    job_name=JOB_NAME,
    job_type=["deploy"],
    walltime="03:30:00", # Réservez pour 1 heure
    env_name="ubuntu2204-min"
).add_machine(
    roles=["master"],
    cluster="paradoxe",
    nodes=1
).add_machine(
    roles=["workers"],
    cluster="paradoxe",
    nodes=2
)
)

print("Définition des ressources terminée.")

Définition des ressources terminée.


# Exécution de la Réservation et du Déploiement

In [3]:

provider = en.G5k(conf)
# Démarrage de la réservation et du déploiement
# provider.init() bloque jusqu'à ce que les noeuds soient prêts

roles, networks = provider.init()

# Récupération des informations sur les rôles pour les cellules suivantes
#roles = reservation.get_roles()
master_node = roles["master"]
worker_nodes = roles["workers"]

print("--- Réservation et déploiement terminés ! ---")
print(f"Master: {[node.address for node in master_node]}")
print(f"Workers: {[node.address for node in worker_nodes]}")
#print(f"Tous les nœuds: roles" + str(roles))

INFO:enoslib.log:[G5k] Reloading 3292534 from rennes
INFO:enoslib.log:[G5k] Waiting for 3292534 on rennes [2025-10-08 08:59:17]
INFO:enoslib.log:[G5k] All jobs are Running !


Output()

Finished 1 tasks (Run dhcp on the nodes) on {'paradoxe-13.rennes.grid5000.fr', 
'paradoxe-26.rennes.grid5000.fr', 'paradoxe-19.rennes.grid5000.fr'}

─────────────────────────────────────────────────────────────────────────────────────────────

--- Réservation et déploiement terminés ! ---
Master: ['paradoxe-13.rennes.grid5000.fr']
Workers: ['paradoxe-19.rennes.grid5000.fr', 'paradoxe-26.rennes.grid5000.fr']


#  Préparation Commune de Tous les Nœuds

In [5]:
# Script de configuration commun
COMMON_SETUP_SCRIPT = """
#!/bin/bash -xe
sudo apt-get update -y
sudo apt-get install -y software-properties-common gpg curl apt-transport-https ca-certificates jq
cat <<EOF | sudo tee /etc/modules-load.d/k8s.conf
overlay
br_netfilter
EOF
sudo modprobe overlay
sudo modprobe br_netfilter
cat <<EOF | sudo tee /etc/sysctl.d/k8s.conf
net.bridge.bridge-nf-call-iptables  = 1
net.bridge.bridge-nf-call-ip6tables = 1
net.ipv4.ip_forward                 = 1
EOF
sudo sysctl --system
sudo swapoff -a
curl -fsSL https://pkgs.k8s.io/addons:/cri-o:/prerelease:/main/deb/Release.key | gpg --dearmor | sudo tee /etc/apt/keyrings/cri-o-apt-keyring.gpg >/dev/null
echo "deb [signed-by=/etc/apt/keyrings/cri-o-apt-keyring.gpg] https://pkgs.k8s.io/addons:/cri-o:/prerelease:/main/deb/ /" | sudo tee /etc/apt/sources.list.d/cri-o.list
sudo apt-get update -y
sudo apt-get install -y cri-o
sudo systemctl enable --now crio
KUBERNETES_VERSION="1.30"
sudo mkdir -p /etc/apt/keyrings
curl -fsSL https://pkgs.k8s.io/core:/stable:/v$KUBERNETES_VERSION/deb/Release.key | sudo gpg --dearmor -o /etc/apt/keyrings/kubernetes-apt-keyring.gpg
echo "deb [signed-by=/etc/apt/keyrings/kubernetes-apt-keyring.gpg] https://pkgs.k8s.io/core:/stable:/v$KUBERNETES_VERSION/deb/ /" | sudo tee /etc/apt/sources.list.d/kubernetes.list
sudo apt-get update
sudo apt-get install -y kubelet=1.30.0-1.1 kubectl=1.30.0-1.1 kubeadm=1.30.0-1.1
sudo apt-mark hold kubelet kubeadm kubectl
IFACE=$(ip route | grep default | awk '{print $5}')
local_ip=$(ip --json addr show $IFACE | jq -r '.[0].addr_info[] | select(.family == "inet") | .local')
echo "KUBELET_EXTRA_ARGS=--node-ip=$local_ip" | sudo tee /etc/default/kubelet
"""

# Exécution du script sur tous les nœuds
print("Lancement de la préparation commune...")
with en.actions(roles=roles) as p:
    p.shell(COMMON_SETUP_SCRIPT)
print("Préparation commune terminée.")

Output()

Lancement de la préparation commune...


Finished 1 tasks (shell) on {'paradoxe-26.rennes.grid5000.fr', 
'paradoxe-19.rennes.grid5000.fr', 'paradoxe-13.rennes.grid5000.fr'}

─────────────────────────────────────────────────────────────────────────────────────────────

Préparation commune terminée.


##  Vérification de la Préparation des Nœuds

In [6]:
# Script de vérification (inchangé)
VERIFY_PREP_SCRIPT = """
#!/bin/bash -e
echo "--- Vérification de l'état des nœuds ---"
echo "[VÉRIF] Service CRI-O (runtime de conteneurs)..."
sudo systemctl is-active --quiet crio
echo "✅ OK: Le service CRI-O est actif."
echo "[VÉRIF] Binaires Kubernetes..."
kubelet --version
kubeadm version
echo "✅ OK: Les binaires Kubelet et Kubeadm sont installés."
echo "[VÉRIF] Modules Kernel..."
lsmod | grep -q br_netfilter && echo "✅ OK: Module 'br_netfilter' est chargé."
lsmod | grep -q overlay && echo "✅ OK: Module 'overlay' est chargé."
echo "\n🎉 La préparation de ce nœud est validée."
"""

# 1. On exécute les actions (comme avant)
print("Lancement de la vérification sur tous les nœuds...")
with en.actions(roles=roles) as p:
    p.shell(VERIFY_PREP_SCRIPT)

results = p.results
verify = results[0].stdout.strip()
print(verify)
# # 2. ✅ NOUVEAU : On récupère et on affiche les résultats APRÈS le bloc
# print("\n--- RÉSULTATS DE LA VÉRIFICATION ---")
# results = p.get_results()

# # On boucle sur les résultats de chaque machine
# for r in results:
#     print(f"\n======== Nœud: {r.host} ========")
#     print(r.stdout) # Affiche la sortie standard de la commande
    
#     # Optionnel : Affiche les erreurs s'il y en a
#     if r.stderr:
#         print(f"-------- Erreurs (stderr) pour {r.host} --------")
#         print(r.stderr)

Output()

Lancement de la vérification sur tous les nœuds...


Finished 1 tasks (shell) on {'paradoxe-26.rennes.grid5000.fr', 
'paradoxe-19.rennes.grid5000.fr', 'paradoxe-13.rennes.grid5000.fr'}

─────────────────────────────────────────────────────────────────────────────────────────────

--- Vérification de l'état des nœuds ---
[VÉRIF] Service CRI-O (runtime de conteneurs)...
✅ OK: Le service CRI-O est actif.
[VÉRIF] Binaires Kubernetes...
Kubernetes v1.30.0
kubeadm version: &version.Info{Major:"1", Minor:"30", GitVersion:"v1.30.0", GitCommit:"7c48c2bd72b9bf5c44d21d7338cc7bea77d0ad2a", GitTreeState:"clean", BuildDate:"2024-04-17T17:34:08Z", GoVersion:"go1.22.2", Compiler:"gc", Platform:"linux/amd64"}
✅ OK: Les binaires Kubelet et Kubeadm sont installés.
[VÉRIF] Modules Kernel...
✅ OK: Module 'br_netfilter' est chargé.
✅ OK: Module 'overlay' est chargé.

🎉 La préparation de ce nœud est validée.


#  Initialisation du Master

In [7]:
# Script d'initialisation du master
MASTER_INIT_SCRIPT = """
#!/bin/bash -xe
IFACE=$(ip route | grep default | awk '{print $5}')
IPADDR=$(ip -4 addr show $IFACE | grep -oP '(?<=inet\s)\d+(\.\d+){3}')
NODENAME=$(hostname -s)
sudo kubeadm init --pod-network-cidr=192.168.0.0/16 --apiserver-advertise-address=$IPADDR --node-name $NODENAME --cri-socket unix:///var/run/crio/crio.sock --ignore-preflight-errors=Swap
mkdir -p $HOME/.kube
sudo cp -i /etc/kubernetes/admin.conf $HOME/.kube/config
sudo chown $(id -u):$(id -g) $HOME/.kube/config
kubectl apply -f https://docs.projectcalico.org/manifests/calico.yaml
"""

# Exécution du script sur le master
print("Initialisation du master...")
with en.actions(roles=master_node) as p:
    p.shell(MASTER_INIT_SCRIPT)
print("Initialisation du master terminée.")

Output()

<>:2: SyntaxWarning: invalid escape sequence '\s'
<>:2: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_1772848/3805460087.py:2: SyntaxWarning: invalid escape sequence '\s'
  MASTER_INIT_SCRIPT = """


Initialisation du master...


Finished 1 tasks (shell) on {'paradoxe-13.rennes.grid5000.fr'}

─────────────────────────────────────────────────────────────────────────────────────────────

Initialisation du master terminée.


# Jonction des Workers

In [8]:
# 1. Récupérer la commande de jonction depuis le master
print("Récupération de la commande de jonction depuis le master...")
with en.actions(roles=master_node) as p:
    p.shell("sudo kubeadm token create --print-join-command")

# Récupération des résultats après le bloc
results = p.results
join_command = results[0].stdout.strip()
print(f"✅ Commande de jonction récupérée: {join_command}")

# 2. Exécuter la commande sur les workers
print("Les workers rejoignent le cluster...")
with en.actions(roles=worker_nodes) as p:
    p.shell(f"sudo {join_command}")

print("Workers joints. Attente de 60s pour la stabilisation...")
time.sleep(60)

Output()

Récupération de la commande de jonction depuis le master...


Finished 1 tasks (shell) on {'paradoxe-13.rennes.grid5000.fr'}

─────────────────────────────────────────────────────────────────────────────────────────────

Output()

✅ Commande de jonction récupérée: kubeadm join 172.16.101.13:6443 --token b6egm9.uq6zcijjozoxlmoj --discovery-token-ca-cert-hash sha256:bad0fee9f4f6d1d960ce22227c751cb142c3711f31b5e40441c1d3ae71c7118c
Les workers rejoignent le cluster...


Finished 1 tasks (shell) on {'paradoxe-26.rennes.grid5000.fr', 
'paradoxe-19.rennes.grid5000.fr'}

─────────────────────────────────────────────────────────────────────────────────────────────

Workers joints. Attente de 60s pour la stabilisation...


# Vérification du Cluster

In [9]:
# Exécution de kubectl get nodes sur le master pour vérifier
with en.actions(roles=master_node) as p:
    p.shell("kubectl get nodes -o wide")

results = p.results
nodes = results[0].stdout.strip()
print("--- État du cluster ---")
print(nodes)

Output()

Finished 1 tasks (shell) on {'paradoxe-13.rennes.grid5000.fr'}

─────────────────────────────────────────────────────────────────────────────────────────────

--- État du cluster ---
NAME          STATUS   ROLES           AGE    VERSION   INTERNAL-IP     EXTERNAL-IP   OS-IMAGE             KERNEL-VERSION       CONTAINER-RUNTIME
paradoxe-13   Ready    control-plane   113s   v1.30.0   172.16.101.13   <none>        Ubuntu 22.04.5 LTS   5.15.0-144-generic   cri-o://1.33.0
paradoxe-19   Ready    <none>          72s    v1.30.0   172.16.101.19   <none>        Ubuntu 22.04.5 LTS   5.15.0-144-generic   cri-o://1.33.0
paradoxe-26   Ready    <none>          72s    v1.30.0   172.16.101.26   <none>        Ubuntu 22.04.5 LTS   5.15.0-144-generic   cri-o://1.33.0


 ## Installation des Add-ons

In [10]:
# Script pour installer les add-ons
ADDONS_SCRIPT = """
#!/bin/bash -e

echo "--- Installation du Metrics Server ---"
kubectl apply -f https://raw.githubusercontent.com/techiescamp/kubeadm-scripts/main/manifests/metrics-server.yaml

echo ""
echo "--- Installation d'Ingress-NGINX ---"
kubectl apply -f https://raw.githubusercontent.com/kubernetes/ingress-nginx/controller-v1.13.0/deploy/static/provider/baremetal/deploy.yaml

echo ""
echo "Installation des add-ons terminée."
"""

# Exécution du script sur le master
print("Installation des add-ons sur le nœud master...")
with en.actions(roles=master_node) as p:
    p.shell(ADDONS_SCRIPT)

# Récupération de la sortie comme vous l'avez spécifié
print("\n--- Résultat de l'installation ---")
if p.results:
    output = p.results[0].stdout.strip()
    print(output)
else:
    print("Aucun résultat à afficher. Vérifiez les erreurs potentielles.")

Output()

Installation des add-ons sur le nœud master...


Finished 1 tasks (shell) on {'paradoxe-13.rennes.grid5000.fr'}

─────────────────────────────────────────────────────────────────────────────────────────────


--- Résultat de l'installation ---
--- Installation du Metrics Server ---
serviceaccount/metrics-server created
clusterrole.rbac.authorization.k8s.io/system:aggregated-metrics-reader created
clusterrole.rbac.authorization.k8s.io/system:metrics-server created
rolebinding.rbac.authorization.k8s.io/metrics-server-auth-reader created
clusterrolebinding.rbac.authorization.k8s.io/metrics-server:system:auth-delegator created
clusterrolebinding.rbac.authorization.k8s.io/system:metrics-server created
service/metrics-server created
deployment.apps/metrics-server created
apiservice.apiregistration.k8s.io/v1beta1.metrics.k8s.io created

--- Installation d'Ingress-NGINX ---
namespace/ingress-nginx created
serviceaccount/ingress-nginx created
serviceaccount/ingress-nginx-admission created
role.rbac.authorization.k8s.io/ingress-nginx created
role.rbac.authorization.k8s.io/ingress-nginx-admission created
clusterrole.rbac.authorization.k8s.io/ingress-nginx created
clusterrole.rbac.authorization.k8s.io

# Déploiement et Test d'une Application

In [11]:
# Script pour déployer l'application de test et son service
APP_DEPLOY_SCRIPT = """
#!/bin/bash -e

echo "--- Création du Deployment et du Service pour l'application echoserver ---"

# On utilise un 'heredoc' pour passer le YAML directement à kubectl
cat <<EOF | kubectl apply -f -
apiVersion: apps/v1
kind: Deployment
metadata:
  name: echoserver
spec:
  replicas: 2
  selector:
    matchLabels:
      app: echoserver
  template:
    metadata:
      labels:
        app: echoserver
    spec:
      containers:
      - name: echoserver
        image: registry.k8s.io/echoserver:1.10
        ports:
        - containerPort: 8080
---
apiVersion: v1
kind: Service
metadata:
  name: echoserver
spec:
  type: NodePort
  selector:
    app: echoserver
  ports:
  - port: 80
    targetPort: 8080
EOF

echo "\n--- Attente de 15 secondes pour le démarrage des pods ---"
sleep 15

echo "\n--- Vérification du statut du déploiement ---"
kubectl get deployment echoserver

echo "\n--- Vérification des pods (devrait en afficher 2) ---"
kubectl get pods -l app=echoserver

echo "\n--- Vérification du service (notez le port après '80:') ---"
kubectl get service echoserver
"""

# Exécution du script sur le master
print("Déploiement de l'application de test...")
with en.actions(roles=master_node) as p:
    p.shell(APP_DEPLOY_SCRIPT)

# Affichage du résultat
print("\n--- Résultat du déploiement ---")
if p.results:
    output = p.results[0].stdout.strip()
    print(output)

Output()

Déploiement de l'application de test...


Finished 1 tasks (shell) on {'paradoxe-13.rennes.grid5000.fr'}

─────────────────────────────────────────────────────────────────────────────────────────────


--- Résultat du déploiement ---
--- Création du Deployment et du Service pour l'application echoserver ---
deployment.apps/echoserver created
service/echoserver created

--- Attente de 15 secondes pour le démarrage des pods ---

--- Vérification du statut du déploiement ---
NAME         READY   UP-TO-DATE   AVAILABLE   AGE
echoserver   2/2     2            2           15s

--- Vérification des pods (devrait en afficher 2) ---
NAME                         READY   STATUS    RESTARTS   AGE
echoserver-d75ff78c5-7mz85   1/1     Running   0          15s
echoserver-d75ff78c5-s9zq6   1/1     Running   0          15s

--- Vérification du service (notez le port après '80:') ---
NAME         TYPE       CLUSTER-IP      EXTERNAL-IP   PORT(S)        AGE
echoserver   NodePort   10.105.247.96   <none>        80:31999/TCP   15s


### Envoyez une Requête

In [13]:
# Récupère l'adresse IP du master
master_ip = master_node[0].address 

# Remplacez 31192 par le port que vous avez obtenu à l'étape 1
node_port = 31999 # ⚠️ CHANGEZ CE PORT

# Test avec curl depuis le master
with en.actions(roles=master_node) as p:
    p.shell(f"curl http://{master_ip}:{node_port}")

if p.results:
    print("--- Réponse du serveur ---")
    print(p.results[0].stdout)

Output()

Finished 1 tasks (shell) on {'paradoxe-13.rennes.grid5000.fr'}

─────────────────────────────────────────────────────────────────────────────────────────────

--- Réponse du serveur ---


Hostname: echoserver-d75ff78c5-7mz85

Pod Information:
	-no pod information available-

Server values:
	server_version=nginx: 1.13.3 - lua: 10008

Request Information:
	client_address=192.168.0.192
	method=GET
	real path=/
	query=
	request_version=1.1
	request_scheme=http
	request_uri=http://paradoxe-13.rennes.grid5000.fr:8080/

Request Headers:
	accept=*/*
	host=paradoxe-13.rennes.grid5000.fr:31999
	user-agent=curl/7.81.0

Request Body:
	-no body in request-


# Installation de la Suite de Monitoring (Prometheus & Grafana)

In [14]:
# Script pour déployer la suite de monitoring
MONITORING_SCRIPT = """
#!/bin/bash -e

echo "--- Installation de la suite Prometheus + Grafana via kube-prometheus ---"

# 1. Cloner le dépôt qui contient les manifestes
if [ ! -d "kube-prometheus" ]; then
    echo "Clonage du dépôt kube-prometheus..."
    git clone https://github.com/prometheus-operator/kube-prometheus.git
else
    echo "Dépôt kube-prometheus déjà présent."
fi
cd kube-prometheus

# 2. Appliquer les définitions de ressources (CRDs) et la configuration de base
echo "Étape 1/3 : Application des CRDs et de la configuration 'setup'..."
kubectl apply --server-side -f manifests/setup

# Attendre que les CRDs soient bien enregistrés dans le cluster avant de continuer
echo "Attente de l'établissement des CRDs..."
kubectl wait --for condition=Established --all CustomResourceDefinition --timeout=300s

# 3. Appliquer le reste de la suite (Prometheus, Grafana, Alertmanager, etc.)
echo "Étape 2/3 : Application des manifestes de la suite de monitoring..."
kubectl apply -f manifests/

# 4. Attendre que les déploiements clés soient prêts dans le namespace 'monitoring'
echo "Étape 3/3 : Attente du démarrage de Prometheus et Grafana (peut prendre quelques minutes)..."
kubectl wait --for=condition=available deployment/prometheus-k8s -n monitoring --timeout=300s
kubectl wait --for=condition=available deployment/grafana -n monitoring --timeout=300s

echo ""
echo "--- Vérification des pods dans le namespace 'monitoring' ---"
kubectl get pods -n monitoring

echo "\n🎉 Suite de monitoring Prometheus et Grafana installée avec succès !"
"""

# Exécution du script sur le master
print("Lancement de l'installation de la suite de monitoring...")
with en.actions(roles=master_node) as p:
    p.shell(MONITORING_SCRIPT)

# Affichage du résultat
print("\n--- Résultat de l'installation ---")
if p.results:
    output = p.results[0].stdout.strip()
    print(output)

Output()

Lancement de l'installation de la suite de monitoring...


Finished 1 tasks (shell) on {'paradoxe-13.rennes.grid5000.fr'}

─────────────────────────────────────────────────────────────────────────────────────────────


--- Résultat de l'installation ---
--- Installation de la suite Prometheus + Grafana via kube-prometheus ---
Clonage du dépôt kube-prometheus...
Étape 1/3 : Application des CRDs et de la configuration 'setup'...
customresourcedefinition.apiextensions.k8s.io/alertmanagerconfigs.monitoring.coreos.com serverside-applied
customresourcedefinition.apiextensions.k8s.io/alertmanagers.monitoring.coreos.com serverside-applied
customresourcedefinition.apiextensions.k8s.io/podmonitors.monitoring.coreos.com serverside-applied
customresourcedefinition.apiextensions.k8s.io/probes.monitoring.coreos.com serverside-applied
customresourcedefinition.apiextensions.k8s.io/prometheuses.monitoring.coreos.com serverside-applied
customresourcedefinition.apiextensions.k8s.io/prometheusagents.monitoring.coreos.com serverside-applied
customresourcedefinition.apiextensions.k8s.io/prometheusrules.monitoring.coreos.com serverside-applied
customresourcedefinition.apiextensions.k8s.io/scrapeconfigs.monitoring.coreos.c

## Vérification des Composants de Monitoring

In [23]:
# Script pour lister les pods et services de monitoring
VERIFY_MONITORING_SCRIPT = """
#!/bin/bash

echo "--- Pods dans le namespace 'monitoring' ---"
kubectl get pods -n monitoring -o wide

echo ""
echo "--- Services dans le namespace 'monitoring' ---"
kubectl get services -n monitoring
echo ""
echo "--- ServiceAccounts dans le namespace 'monitoring' ---"
kubectl get serviceaccounts -n monitoring
"""

# Exécution du script de vérification sur le master
print("Récupération de l'état des composants de monitoring...")
with en.actions(roles=master_node) as p:
    p.shell(VERIFY_MONITORING_SCRIPT)

# Affichage du résultat
if p.results:
    output = p.results[0].stdout.strip()
    print(output)

Output()

Récupération de l'état des composants de monitoring...


Finished 1 tasks (shell) on {'paradoxe-13.rennes.grid5000.fr'}

─────────────────────────────────────────────────────────────────────────────────────────────

--- Pods dans le namespace 'monitoring' ---
NAME                                   READY   STATUS    RESTARTS   AGE   IP              NODE          NOMINATED NODE   READINESS GATES
alertmanager-main-0                    2/2     Running   0          39m   192.168.64.70   paradoxe-19   <none>           <none>
alertmanager-main-1                    2/2     Running   0          39m   192.168.64.8    paradoxe-26   <none>           <none>
alertmanager-main-2                    2/2     Running   0          39m   192.168.64.7    paradoxe-26   <none>           <none>
blackbox-exporter-77966b4779-n2g89     3/3     Running   0          39m   192.168.64.3    paradoxe-26   <none>           <none>
grafana-5997747455-cxs9s               1/1     Running   0          39m   192.168.64.68   paradoxe-19   <none>           <none>
kube-state-metrics-756cd6cb9d-wrwrl    3/3     Running   0          39m   192.168.64.4    paradoxe-26   <none>           <none>
node-exporter-2fxqc                    2/2     Runn

## Mise à Jour des NetworkPolicies pour l'Accès Externe

In [37]:
# Script pour mettre à jour les NetworkPolicies
UPDATE_NP_SCRIPT = """
#!/bin/bash -e

echo "--- 1. Mise à jour de la NetworkPolicy de Grafana ---"
cat <<EOF | kubectl apply -f -
apiVersion: networking.k8s.io/v1
kind: NetworkPolicy
metadata:
  name: grafana
  namespace: monitoring
  labels:
    app.kubernetes.io/component: grafana
    app.kubernetes.io/name: grafana
    app.kubernetes.io/part-of: kube-prometheus
    app.kubernetes.io/version: 12.2.0
spec:
  podSelector:
    matchLabels:
      app.kubernetes.io/component: grafana
      app.kubernetes.io/name: grafana
      app.kubernetes.io/part-of: kube-prometheus
  policyTypes:
  - Ingress
  - Egress
  ingress:
  - from:
    # On autorise le trafic depuis n'importe quelle adresse IP (externe/interne)
    - ipBlock:
        cidr: 0.0.0.0/0
    # On garde la règle existante qui autorise Prometheus
    - podSelector:
        matchLabels:
          app.kubernetes.io/name: prometheus
    ports:
    - port: 3000
      protocol: TCP
  egress:
  - {}
EOF

echo "\n--- 2. Mise à jour de la NetworkPolicy de Prometheus ---"
cat <<EOF | kubectl apply -f -
apiVersion: networking.k8s.io/v1
kind: NetworkPolicy
metadata:
  name: prometheus-k8s
  namespace: monitoring
  labels:
    app.kubernetes.io/component: prometheus
    app.kubernetes.io/instance: k8s
    app.kubernetes.io/name: prometheus
    app.kubernetes.io/part-of: kube-prometheus
    app.kubernetes.io/version: 3.6.0
spec:
  podSelector:
    matchLabels:
      app.kubernetes.io/component: prometheus
      app.kubernetes.io/instance: k8s
      app.kubernetes.io/name: prometheus
      app.kubernetes.io/part-of: kube-prometheus
  policyTypes:
  - Ingress
  - Egress
  ingress:
  - from:
    # On autorise le trafic depuis n'importe quelle adresse IP
    - ipBlock:
        cidr: 0.0.0.0/0
    # On garde les règles existantes pour la communication interne
    - namespaceSelector: {}
    - podSelector:
        matchLabels:
          app.kubernetes.io/name: prometheus
    - podSelector:
        matchLabels:
          app.kubernetes.io/name: prometheus-adapter
    - podSelector:
        matchLabels:
          app.kubernetes.io/name: grafana
    ports:
    - port: 9090
      protocol: TCP
    - port: 8080
      protocol: TCP
  egress:
  - {}
EOF

echo "\nNetworkPolicies mises à jour avec succès pour autoriser l'accès externe."
"""

# Exécution du script sur le master
print("Mise à jour des NetworkPolicies...")
with en.actions(roles=master_node) as p:
    p.shell(UPDATE_NP_SCRIPT)

# Affichage du résultat
if p.results:
    output = p.results[0].stdout.strip()
    print(output)

Output()

Mise à jour des NetworkPolicies...


Finished 1 tasks (shell) on {'paradoxe-13.rennes.grid5000.fr'}

─────────────────────────────────────────────────────────────────────────────────────────────

--- 1. Mise à jour de la NetworkPolicy de Grafana ---
networkpolicy.networking.k8s.io/grafana unchanged

--- 2. Mise à jour de la NetworkPolicy de Prometheus ---
networkpolicy.networking.k8s.io/prometheus-k8s configured

NetworkPolicies mises à jour avec succès pour autoriser l'accès externe.


## Passage des Services en NodePort

In [17]:
# Script pour patcher les services en NodePort
PATCH_SERVICES_SCRIPT = """
#!/bin/bash -e

echo "--- 1. Modification du service Grafana en type NodePort ---"
kubectl patch service grafana -n monitoring -p '{"spec": {"type": "NodePort"}}'

echo ""
echo "--- 2. Modification du service Prometheus en type NodePort ---"
kubectl patch service prometheus-k8s -n monitoring -p '{"spec": {"type": "NodePort"}}'

echo ""
echo "--- 3. Vérification du résultat ---"
# Petite pause pour s'assurer que l'API server a bien traité les changements
sleep 2
kubectl get services -n monitoring
"""

# Exécution du script sur le master
print("Patch des services Grafana et Prometheus en NodePort...")
with en.actions(roles=master_node) as p:
    p.shell(PATCH_SERVICES_SCRIPT)

# Affichage du résultat
print("\n--- Résultat de la commande patch ---")
if p.results:
    output = p.results[0].stdout.strip()
    print(output)

Output()

Patch des services Grafana et Prometheus en NodePort...


Finished 1 tasks (shell) on {'paradoxe-13.rennes.grid5000.fr'}

─────────────────────────────────────────────────────────────────────────────────────────────


--- Résultat de la commande patch ---
--- 1. Modification du service Grafana en type NodePort ---
service/grafana patched

--- 2. Modification du service Prometheus en type NodePort ---
service/prometheus-k8s patched

--- 3. Vérification du résultat ---
NAME                    TYPE        CLUSTER-IP       EXTERNAL-IP   PORT(S)                         AGE
alertmanager-main       ClusterIP   10.107.218.114   <none>        9093/TCP,8080/TCP               79s
alertmanager-operated   ClusterIP   None             <none>        9093/TCP,9094/TCP,9094/UDP      62s
blackbox-exporter       ClusterIP   10.109.154.230   <none>        9115/TCP,19115/TCP              79s
grafana                 NodePort    10.101.166.48    <none>        3000:30877/TCP                  78s
kube-state-metrics      ClusterIP   None             <none>        8443/TCP,9443/TCP               78s
node-exporter           ClusterIP   None             <none>        9100/TCP                        77s
prometheus-adapter      

#  Installation de Kepler pour le Monitoring Énergétique

In [18]:
import os

# Script mis à jour avec le bon paramètre pour le ServiceMonitor
INSTALL_KEPLER_SCRIPT = """
#!/bin/bash -e

# --- 1. Installation de Helm (si non présent) ---
if ! command -v helm &> /dev/null
then
    echo "Helm non trouvé. Installation de Helm..."
    curl -fsSL -o get_helm.sh https://raw.githubusercontent.com/helm/helm/main/scripts/get-helm-3
    chmod 700 get_helm.sh
    ./get_helm.sh
else
    echo "Helm est déjà installé."
fi

# --- 2. Ajout du dépôt Helm de Kepler ---
echo ""
echo "--- Ajout et mise à jour du dépôt Helm de Kepler ---"
helm repo add kepler https://sustainable-computing-io.github.io/kepler-helm-chart
helm repo update

# --- 3. Installation ou Mise à Jour de Kepler ---
echo ""
echo "--- Déploiement/Mise à jour de Kepler dans le namespace 'kepler' ---"
# CORRECTION : On utilise 'serviceMonitor.enabled=true'
helm upgrade --install kepler kepler/kepler --namespace kepler --create-namespace \
    --set serviceMonitor.enabled=true \
    --set service.type=NodePort

# --- 4. Vérification de l'installation ---
echo ""
echo "--- Attente des composants Kepler ---"
kubectl wait --for=condition=ready pod -l app.kubernetes.io/name=kepler -n kepler --timeout=300s

echo ""
echo "--- Vérification des pods Kepler ---"
kubectl get pods -n kepler

echo ""
echo "--- Vérification que le ServiceMonitor a bien été créé ---"
kubectl get servicemonitor -n kepler

echo "--- Vérification que le service est bien de type NodePort ---"
kubectl get service -n kepler

echo ""
echo "🎉 Kepler est maintenant correctement configuré avec un ServiceMonitor et un NodePort !"
"""

# --- Étape 1 : Enregistrer le script dans un fichier local ---
script_path = "install_kepler.sh"
with open(script_path, "w") as f:
    f.write(INSTALL_KEPLER_SCRIPT)
os.chmod(script_path, 0o755)
print(f"Le script d'installation final a été sauvegardé dans '{script_path}'.")


# --- Étape 2 : Exécuter le fichier script avec enoslib ---
print("Lancement de la mise à jour de Kepler...")
with en.actions(roles=master_node) as p:
    p.script(script_path)

# Affichage du résultat
if p.results:
    output = p.results[0].stdout.strip()
    print(output)

Output()

Le script d'installation final a été sauvegardé dans 'install_kepler.sh'.
Lancement de la mise à jour de Kepler...


Finished 1 tasks (script) on {'paradoxe-13.rennes.grid5000.fr'}

─────────────────────────────────────────────────────────────────────────────────────────────

Helm non trouvé. Installation de Helm...
Verifying checksum... Done.
Preparing to install helm into /usr/local/bin
helm installed into /usr/local/bin/helm

--- Ajout et mise à jour du dépôt Helm de Kepler ---
"kepler" has been added to your repositories
Hang tight while we grab the latest from your chart repositories...
...Successfully got an update from the "kepler" chart repository
Update Complete. ⎈Happy Helming!⎈

--- Déploiement/Mise à jour de Kepler dans le namespace 'kepler' ---
Release "kepler" does not exist. Installing it now.
NAME: kepler
LAST DEPLOYED: Wed Oct  8 09:12:34 2025
NAMESPACE: kepler
STATUS: deployed
REVISION: 1
TEST SUITE: None

--- Attente des composants Kepler ---
pod/kepler-bdhn6 condition met
pod/kepler-c4s57 condition met
pod/kepler-r2xkl condition met

--- Vérification des pods Kepler ---
NAME           READY   STATUS    RESTARTS   AGE
kepler-bdhn6   1/1     Running   0          20s
kepler-c4s57   1/1     Running   0          20s
kepler-r2xkl   1/1     Run

## Vérification des ServiceMonitors

In [67]:
# Script to list ServiceMonitors
CHECK_SM_SCRIPT = """
#!/bin/bash

echo "--- Recherche des ServiceMonitors dans tous les namespaces (-A) ---"
kubectl get servicemonitor -A
"""

# Execute the verification script on the master node
print("Checking for ServiceMonitor resources across the cluster...")
with en.actions(roles=master_node) as p:
    p.shell(CHECK_SM_SCRIPT)

# Display the result
if p.results:
    output = p.results[0].stdout.strip()
    print(output)

Output()

Checking for ServiceMonitor resources across the cluster...


Finished 1 tasks (shell) on {'paradoxe-5.rennes.grid5000.fr'}

─────────────────────────────────────────────────────────────────────────────────────────────

--- Recherche des ServiceMonitors dans tous les namespaces (-A) ---
NAMESPACE    NAME                         AGE
kepler       kepler-prometheus-exporter   39s
monitoring   alertmanager-main            2m39s
monitoring   blackbox-exporter            2m39s
monitoring   coredns                      2m38s
monitoring   grafana                      2m38s
monitoring   kube-apiserver               2m38s
monitoring   kube-controller-manager      2m38s
monitoring   kube-scheduler               2m38s
monitoring   kube-state-metrics           2m38s
monitoring   kubelet                      2m38s
monitoring   node-exporter                2m38s
monitoring   prometheus-adapter           2m37s
monitoring   prometheus-k8s               2m38s
monitoring   prometheus-operator          2m37s


## Suppression Complète de Kepler

In [19]:
# Script pour supprimer complètement Kepler
UNINSTALL_KEPLER_SCRIPT = """
#!/bin/bash

echo "--- 1. Désinstallation de la release Helm 'kepler' ---"
# On vérifie si la release existe avant de tenter de la supprimer
if helm status kepler -n kepler &> /dev/null; then
    helm uninstall kepler -n kepler
else
    echo "Release Helm 'kepler' non trouvée."
fi

echo "\n--- 2. Suppression du ServiceMonitor dans le namespace 'monitoring' ---"
# --ignore-not-found=true évite une erreur si la ressource a déjà été supprimée
kubectl delete servicemonitor kepler-prometheus-exporter -n monitoring --ignore-not-found=true

echo "\n--- 3. Suppression du namespace 'kepler' ---"
kubectl delete namespace kepler --ignore-not-found=true

echo "\n--- 4. Suppression du dépôt Helm de la configuration locale ---"
if helm repo list | grep -q "kepler"; then
    helm repo remove kepler
else
    echo "Dépôt Helm 'kepler' non trouvé."
fi

echo "\n🎉 Nettoyage de Kepler terminé."
"""

# Exécution du script de suppression sur le master
print("Lancement de la suppression complète de Kepler...")
with en.actions(roles=master_node) as p:
    p.shell(UNINSTALL_KEPLER_SCRIPT)

if p.results:
    output = p.results[0].stdout.strip()
    print(output)

Output()

Lancement de la suppression complète de Kepler...


Finished 1 tasks (shell) on {'paradoxe-13.rennes.grid5000.fr'}

─────────────────────────────────────────────────────────────────────────────────────────────

--- 1. Désinstallation de la release Helm 'kepler' ---
NAME: kepler
LAST DEPLOYED: Wed Oct  8 09:12:34 2025
NAMESPACE: kepler
STATUS: deployed
REVISION: 1
TEST SUITE: None
release "kepler" uninstalled

--- 2. Suppression du ServiceMonitor dans le namespace 'monitoring' ---

--- 3. Suppression du namespace 'kepler' ---
namespace "kepler" deleted

--- 4. Suppression du dépôt Helm de la configuration locale ---
"kepler" has been removed from your repositories

🎉 Nettoyage de Kepler terminé.


# Installer Kepler depuis les Manifestes Locaux

In [22]:
import os

# --- Étape 1 : Copier le répertoire local 'kepler' vers le master ---

source_dir = "kepler"
dest_dir = "/tmp/kepler"

if not os.path.isdir(source_dir):
    raise FileNotFoundError(f"Le répertoire local '{source_dir}' est introuvable.")

print(f"--- Copie du répertoire local '{source_dir}' vers '{dest_dir}' sur le master... ---")
with en.actions(roles=master_node) as p:
    p.copy(src=source_dir, dest="/tmp/")

if p.results and p.results[0].status == "OK":
    print("Répertoire copié avec succès.")
else:
    print("Erreur lors de la copie du répertoire.")
    if p.results:
        print(p.results[0].stderr)


# --- Étape 2 : Appliquer les manifestes et vérifier ---

# Script simplifié : la création du namespace est gérée par les manifestes
APPLY_KEPLER_MANIFESTS_SCRIPT = f"""
#!/bin/bash -e

echo "--- Application de tous les manifestes depuis le répertoire {dest_dir} ---"
kubectl apply -f "{dest_dir}"

echo ""
echo "--- Attente de 30 secondes pour le démarrage des pods ---"
sleep 30

echo ""
echo "--- Vérification du statut des pods dans le namespace 'kepler' ---"
kubectl get pods -n kepler

echo "--- Vérification que le Service a bien été créé ---"
kubectl get svc -n kepler -o wide

echo ""
echo "🎉 Kepler a été déployé à partir des manifestes locaux."
"""

print(f"\n--- Application des manifestes sur le cluster... ---")
with en.actions(roles=master_node) as p:
    p.shell(APPLY_KEPLER_MANIFESTS_SCRIPT)

if p.results:
    output = p.results[0].stdout.strip()
    print(output)

Output()

--- Copie du répertoire local 'kepler' vers '/tmp/kepler' sur le master... ---


Finished 1 tasks (copy) on {'paradoxe-13.rennes.grid5000.fr'}

─────────────────────────────────────────────────────────────────────────────────────────────

Output()

Répertoire copié avec succès.

--- Application des manifestes sur le cluster... ---


Finished 1 tasks (shell) on {'paradoxe-13.rennes.grid5000.fr'}

─────────────────────────────────────────────────────────────────────────────────────────────

--- Application de tous les manifestes depuis le répertoire /tmp/kepler ---
configmap/kepler unchanged
daemonset.apps/kepler unchanged
namespace/kepler unchanged
role.rbac.authorization.k8s.io/prom-kepler unchanged
rolebinding.rbac.authorization.k8s.io/prom-kepler unchanged
serviceaccount/kepler unchanged
clusterrole.rbac.authorization.k8s.io/kepler unchanged
clusterrolebinding.rbac.authorization.k8s.io/kepler unchanged
service/kepler unchanged
servicemonitor.monitoring.coreos.com/kepler unchanged

--- Attente de 30 secondes pour le démarrage des pods ---

--- Vérification du statut des pods dans le namespace 'kepler' ---
NAME           READY   STATUS    RESTARTS   AGE
kepler-6hv64   1/1     Running   0          98s
kepler-8nqbt   1/1     Running   0          98s
kepler-tdxsq   1/1     Running   0          98s
--- Vérification que le Service a bien été créé ---
NAME     TYPE       CLUSTER-IP      EXTERNAL-IP   PORT(S)           AGE     SELECTOR
kepler   NodePort   10.111.96.169   <none

## Diagnostic - Pourquoi Prometheus ne Scrape pas Kepler ?

In [38]:
# Script de diagnostic complet pour identifier pourquoi Prometheus ne scrape pas Kepler
DIAGNOSTIC_KEPLER_PROMETHEUS = """
#!/bin/bash

echo "======================================================================"
echo "🔍 DIAGNOSTIC COMPLET: Prometheus ↔ Kepler"
echo "======================================================================"

echo ""
echo "--- 1. Vérification des Pods Kepler ---"
kubectl get pods -n kepler -o wide

echo ""
echo "--- 2. Vérification du Service Kepler ---"
kubectl get svc -n kepler -o wide

echo ""
echo "--- 3. Vérification des Labels du Service Kepler ---"
echo "Les labels sont CRITIQUES pour que le ServiceMonitor fonctionne!"
kubectl get svc -n kepler -o yaml | grep -A 10 "labels:"

echo ""
echo "--- 4. Vérification du ServiceMonitor Kepler ---"
kubectl get servicemonitor -n kepler -o wide

echo ""
echo "--- 5. Détails du ServiceMonitor (selector et labels) ---"
kubectl get servicemonitor -n kepler -o yaml | grep -A 20 "selector:"

echo ""
echo "--- 6. Vérification que Prometheus-Operator voit le ServiceMonitor ---"
echo "Prometheus doit être dans le même namespace ou avec les bonnes règles RBAC"
kubectl get servicemonitor -A

echo ""
echo "--- 7. Configuration de Prometheus pour ServiceMonitor ---"
echo "Vérification des serviceMonitorSelector dans Prometheus:"
kubectl get prometheus -n monitoring -o yaml | grep -A 5 "serviceMonitorSelector:"

echo ""
echo "--- 8. Vérification des Endpoints Kepler ---"
echo "Si pas d'endpoints, le service ne pointe vers rien!"
kubectl get endpoints -n kepler

echo ""
echo "--- 9. Test direct du endpoint Kepler ---"
KEPLER_POD=`kubectl get pods -n kepler -l app.kubernetes.io/name=kepler -o jsonpath='{.items[0].metadata.name}'`
if [ -n "$KEPLER_POD" ]; then
    echo "Test de l'endpoint metrics sur le pod $KEPLER_POD:"
    kubectl exec -n kepler $KEPLER_POD -- curl -s localhost:9102/metrics | head -20
else
    echo "❌ Aucun pod Kepler trouvé!"
fi

echo ""
echo "--- 10. Vérification des Logs Prometheus ---"
echo "Recherche d'erreurs liées à Kepler dans les logs Prometheus:"
PROM_POD=`kubectl get pods -n monitoring -l app.kubernetes.io/name=prometheus -o jsonpath='{.items[0].metadata.name}'`
if [ -n "$PROM_POD" ]; then
    kubectl logs -n monitoring $PROM_POD -c prometheus --tail=50 | grep -i kepler || echo "Aucune mention de Kepler dans les logs récents"
else
    echo "❌ Pod Prometheus non trouvé!"
fi

echo ""
echo "--- 11. Configuration Actuelle des Targets Prometheus ---"
echo "Vérifiez si Kepler apparaît dans les targets de Prometheus"
echo "💡 Accédez à l'UI Prometheus: Status → Targets"
echo "💡 Ou utilisez l'API: curl http://prometheus-ip:port/api/v1/targets"

echo ""
echo "======================================================================"
echo "🎯 CAUSES COMMUNES ET SOLUTIONS:"
echo "======================================================================"
echo ""
echo "❌ PROBLÈME 1: Labels incompatibles"
echo "   Le ServiceMonitor cherche des labels spécifiques sur le Service"
echo "   Solution: Vérifiez que les labels du Service matchent le selector du ServiceMonitor"
echo ""
echo "❌ PROBLÈME 2: Namespace incorrect"
echo "   Le ServiceMonitor doit être dans un namespace que Prometheus surveille"
echo "   Solution: Soit mettre le ServiceMonitor dans 'monitoring', soit configurer Prometheus"
echo ""
echo "❌ PROBLÈME 3: serviceMonitorSelector vide"
echo "   Prometheus n'est configuré pour surveiller AUCUN ServiceMonitor"
echo "   Solution: Configurer prometheus.serviceMonitorSelector: {}"
echo ""
echo "❌ PROBLÈME 4: Port incorrect"
echo "   Le ServiceMonitor pointe vers un port qui n'existe pas"
echo "   Solution: Vérifier que le port dans ServiceMonitor = port du Service"
echo ""
echo "======================================================================"
"""

# --- Enregistrer le script dans un fichier local ---
script_path = "diagnostic_kepler.sh"
with open(script_path, "w") as f:
    f.write(DIAGNOSTIC_KEPLER_PROMETHEUS)
os.chmod(script_path, 0o755)

print("🔍 Lancement du diagnostic complet Prometheus ↔ Kepler...")
with en.actions(roles=master_node) as p:
    p.script(script_path)

if p.results:
    output = p.results[0].stdout.strip()
    print(output)

Output()

🔍 Lancement du diagnostic complet Prometheus ↔ Kepler...


Finished 1 tasks (script) on {'paradoxe-13.rennes.grid5000.fr'}

─────────────────────────────────────────────────────────────────────────────────────────────

🔍 DIAGNOSTIC COMPLET: Prometheus ↔ Kepler

--- 1. Vérification des Pods Kepler ---
NAME           READY   STATUS    RESTARTS   AGE   IP              NODE          NOMINATED NODE   READINESS GATES
kepler-6hv64   1/1     Running   0          47m   192.168.64.10   paradoxe-26   <none>           <none>
kepler-8nqbt   1/1     Running   0          47m   192.168.0.196   paradoxe-13   <none>           <none>
kepler-tdxsq   1/1     Running   0          47m   192.168.64.72   paradoxe-19   <none>           <none>

--- 2. Vérification du Service Kepler ---
NAME     TYPE       CLUSTER-IP      EXTERNAL-IP   PORT(S)           AGE   SELECTOR
kepler   NodePort   10.111.96.169   <none>        28282:31515/TCP   48m   app.kubernetes.io/name=kepler,app.kubernetes.io/part-of=kepler

--- 3. Vérification des Labels du Service Kepler ---
Les labels sont CRITIQUES pour que le ServiceMonitor fonctionne!
    labels:
      app.kubernetes.io/name: kepler
      app.kubernetes.io/part-of: kepler
    name: kepler
    

## 🔧 Solution : Donner les Permissions RBAC à Prometheus pour Kepler

In [39]:
# Script pour corriger les permissions RBAC de Prometheus
FIX_PROMETHEUS_RBAC = """
#!/bin/bash -e

echo "========================================================================"
echo "🔧 Correction des Permissions RBAC pour Prometheus → Kepler"
echo "========================================================================"

echo ""
echo "--- Création d'un ClusterRole avec les permissions nécessaires ---"
cat <<EOF | kubectl apply -f -
apiVersion: rbac.authorization.k8s.io/v1
kind: ClusterRole
metadata:
  name: prometheus-kepler-access
rules:
- apiGroups: [""]
  resources:
  - services
  - endpoints
  - pods
  verbs: ["get", "list", "watch"]
- apiGroups: ["discovery.k8s.io"]
  resources:
  - endpointslices
  verbs: ["get", "list", "watch"]
EOF

echo ""
echo "--- Liaison du ClusterRole au ServiceAccount prometheus-k8s ---"
cat <<EOF | kubectl apply -f -
apiVersion: rbac.authorization.k8s.io/v1
kind: ClusterRoleBinding
metadata:
  name: prometheus-kepler-access
roleRef:
  apiGroup: rbac.authorization.k8s.io
  kind: ClusterRole
  name: prometheus-kepler-access
subjects:
- kind: ServiceAccount
  name: prometheus-k8s
  namespace: monitoring
EOF

echo ""
echo "--- Vérification des permissions créées ---"
kubectl get clusterrole prometheus-kepler-access
kubectl get clusterrolebinding prometheus-kepler-access

echo ""
echo "--- Redémarrage des pods Prometheus pour appliquer les changements ---"
kubectl rollout restart statefulset prometheus-k8s -n monitoring

echo ""
echo "--- Attente du redémarrage de Prometheus (30 secondes) ---"
sleep 30

echo ""
echo "--- Vérification que Prometheus est prêt ---"
kubectl wait --for=condition=ready pod -l app.kubernetes.io/name=prometheus -n monitoring --timeout=120s

echo ""
echo "========================================================================"
echo "✅ Permissions RBAC configurées avec succès !"
echo "========================================================================"
echo ""
echo "Prometheus peut maintenant accéder aux EndpointSlices du namespace kepler."
echo "Attendez 1-2 minutes puis vérifiez les targets dans l'UI Prometheus."
"""

print("🔧 Application de la correction RBAC...")
with en.actions(roles=master_node) as p:
    p.shell(FIX_PROMETHEUS_RBAC)

if p.results:
    output = p.results[0].stdout.strip()
    print(output)

Output()

🔧 Application de la correction RBAC...


Finished 1 tasks (shell) on {'paradoxe-13.rennes.grid5000.fr'}

─────────────────────────────────────────────────────────────────────────────────────────────

🔧 Correction des Permissions RBAC pour Prometheus → Kepler

--- Création d'un ClusterRole avec les permissions nécessaires ---
clusterrole.rbac.authorization.k8s.io/prometheus-kepler-access created

--- Liaison du ClusterRole au ServiceAccount prometheus-k8s ---
clusterrolebinding.rbac.authorization.k8s.io/prometheus-kepler-access created

--- Vérification des permissions créées ---
NAME                       CREATED AT
prometheus-kepler-access   2025-10-08T08:31:22Z
NAME                       ROLE                                   AGE
prometheus-kepler-access   ClusterRole/prometheus-kepler-access   0s

--- Redémarrage des pods Prometheus pour appliquer les changements ---
statefulset.apps/prometheus-k8s restarted

--- Attente du redémarrage de Prometheus (30 secondes) ---

--- Vérification que Prometheus est prêt ---
pod/prometheus-k8s-0 condition met
pod/prometheus-k8s-1 condition met

✅ Permissions RBAC configurées avec succès !

Prometheus peut maintenant accéder aux EndpointSlice

# # Libération des Ressources

In [82]:
# Destruction de la réservation
print("Libération des ressources sur Grid'5000...")
provider.destroy()
print("Ressources libérées. ✅")

Libération des ressources sur Grid'5000...


INFO:enoslib.log:[G5k] Reloading 3292322 from rennes
INFO:enoslib.log:[G5k] Killing the job (rennes, 3292322)
INFO:enoslib.log:[G5k] Killing the job (rennes, 3292322)
INFO:enoslib.log:[G5k] Job killed (rennes, 3292322)
INFO:enoslib.log:[G5k] Job killed (rennes, 3292322)


Ressources libérées. ✅
